# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LetterNumber
df = pd.read_csv('LetterNumber_with_MethylomeDetails.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LetterNumber
(458, 73193)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,LetterNumber_Category,cg20815302,cg25795414,...,cg27475074,cg02546039,cg25870071,cg26765773,cg22019496,cg02429154,cg05411695,cg13202877,cg17697230,cg05687264
0,200973410159_R03C01,3000,2,2,0,69.1,4,1,0.915790,0.899909,...,0.028559,0.920549,0.469715,0.112805,0.892764,0.925107,0.070574,0.165807,0.918808,0.928897
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.907957,0.868910,...,0.049168,0.906285,0.458602,0.068597,0.859078,0.907987,0.036712,0.177144,0.927030,0.911733
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.914788,0.898327,...,0.023227,0.902970,0.345503,0.064245,0.873081,0.928342,0.046664,0.176564,0.909178,0.932602
3,200989060105_R03C01,3003,2,1,2,56.7,3,1,0.932803,0.871871,...,0.023789,0.897105,0.433373,0.073856,0.894786,0.923004,0.049485,0.146706,0.935477,0.916588
4,200989060135_R07C01,3004,1,2,0,59.4,3,1,0.915434,0.883100,...,0.028658,0.913353,0.459784,0.054531,0.899957,0.911859,0.037488,0.155276,0.899873,0.906124


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'LetterNumber_Category','Age','AgeCate'], axis=1)
print(df1.shape)
df1.head(2)

(458, 73186)


,APPRDX,cg20815302,cg25795414,cg00959375,cg07781847,cg05138383,cg27394794,cg12443990,cg19788329,cg22178513,...,cg27475074,cg02546039,cg25870071,cg26765773,cg22019496,cg02429154,cg05411695,cg13202877,cg17697230,cg05687264
0,2,0.915790,0.899909,0.804116,0.796131,0.740029,0.838002,0.822036,0.813254,0.745601,...,0.028559,0.920549,0.469715,0.112805,0.892764,0.925107,0.070574,0.165807,0.918808,0.928897
1,1,0.907957,0.868910,0.752844,0.827413,0.771393,0.766283,0.791932,0.785104,0.723989,...,0.049168,0.906285,0.458602,0.068597,0.859078,0.907987,0.036712,0.177144,0.927030,0.911733


## RanFor 100 times, CpG only 

In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [5]:
dframe1.to_csv ("LetterNumber_AllCpG_Accuracy_100RF.csv", index=False)

In [6]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.565,0.111915
1,0.515,0.148408


## RanFor 1000 times. Samples = 50

In [7]:
df2 = df1.drop (['APPRDX'], axis=1)

In [8]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [9]:
print(dframe1.head(3))
dframe1.to_csv ("LetterNumber_Rand50CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.490  0.104403
1     0.450  0.120416
2     0.455  0.101119


## RanFor 1000 times. Samples = 4

In [10]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(4, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [11]:
print(dframe1.head(3))
dframe1.to_csv ("LetterNumber_Rand4CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.490  0.104403
1     0.450  0.120416
2     0.455  0.101119
